In [15]:
from keras.layers import Input, Dense, Lambda, Flatten, Activation, Merge, Concatenate, Add
from keras import layers
from keras.layers.merge import concatenate
from keras.models import Model, Sequential
from keras.objectives import binary_crossentropy
from keras.callbacks import LearningRateScheduler

import numpy as np

from keras import backend as K
from keras.models import load_model
import tensorflow as tf

from models import vgg
from models.cvae import CVAE
from utils.angles import deg2bit, bit2deg
from utils.losses import mad_loss_tf, cosine_loss_tf, von_mises_loss_tf, maad_from_deg
from utils.losses import gaussian_kl_divergence_tf, gaussian_kl_divergence_np
from utils.losses  import von_mises_log_likelihood_tf, von_mises_log_likelihood_np
from utils.towncentre import load_towncentre
from utils.experiements import get_experiment_id

In [25]:
xtr, ytr_deg, xval, yval_deg, xte, yte_deg = load_towncentre('data/TownCentre.pkl.gz', canonical_split=True)
image_height, image_width = xtr.shape[1], xtr.shape[2]
ytr_bit = deg2bit(ytr_deg)
yval_bit = deg2bit(yval_deg)
yte_bit = deg2bit(yte_deg)

image_height, image_width, n_channels = xtr.shape[1:]
flatten_x_shape = xtr[0].flatten().shape[0]
phi_shape = yte_bit.shape[1]

In [28]:
custom_objects = {}
custom_objects.update({"cosine_loss_tf":von_mises_loss_tf})

In [29]:
vgg_von_mises = load_model('/Users/sergey/BiternionNet/logs/vggbit_likelihood_fixed_kappa_Adadelta_200epochs_batch10_DEBUG_213a35d4d583678c404d/vgg_bit_cosine_town.best_model.h5',
                          custom_objects=custom_objects)

In [30]:
vgg_von_mises.predict(xtr)

array([[-0.99905026, -0.04357499],
       [-0.99444807, -0.10522822],
       [-0.99633926, -0.08548737],
       ..., 
       [-0.99898928,  0.04494935],
       [-0.96684736, -0.255355  ],
       [-0.99892819, -0.04628797]], dtype=float32)